In [26]:
import pandas as pd
import utils as ut
import ast
import importlib

In [27]:
importlib.reload(ut)

<module 'utils' from '/home/mauro/HENRY FT17/Proyectos/PI 1/Steam-Rec-System/ETL/utils.py'>

In [31]:
# Especifica la ruta del archivo JSON
json_path = "../data/originals/user_reviews.json"

reviews = []
with open(json_path, 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))

reviews_df = pd.DataFrame(reviews)



#### The first record is reviewed to see its content and the names of the columns.

In [32]:
print(reviews_df.iloc[0])

user_id                                     76561197970982479
user_url    http://steamcommunity.com/profiles/76561197970...
reviews     [{'funny': '', 'posted': 'Posted November 5, 2...
Name: 0, dtype: object


In [33]:
# This step is applied to convert the lists into strings and thus be able to generate an overview of the data.
reviews_df['reviews'] = reviews_df['reviews'].apply(str)

In [34]:
ut.data_overview(reviews_df)


Total rows:  25799

Total full null rows:  0

Total duplicated rows: 313


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,user_id,[<class 'str'>],100.0,25799,0.0,0
1,user_url,[<class 'str'>],100.0,25799,0.0,0
2,reviews,[<class 'str'>],100.0,25799,0.0,0


#### There are no nulls, so we proceed to remove duplicates.

In [35]:
reviews_df.drop_duplicates(inplace=True)

In [36]:
ut.data_overview(reviews_df)


Total rows:  25486

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,user_id,[<class 'str'>],100.0,25486,0.0,0
1,user_url,[<class 'str'>],100.0,25486,0.0,0
2,reviews,[<class 'str'>],100.0,25486,0.0,0


#### The presence of string values 'None' in the columns is checked.

In [37]:
ut.check_none_values(reviews_df)

    Columna  Porcentaje None
0   user_id              0.0
1  user_url              0.0
2   reviews              0.0


#### Basic ETL completed, data types have been successfully adjusted, and there are no null values or duplicates. It is exported to CSV to facilitate the subsequent handling of the dataset.


##### The column 'reviews,'contain a list with json dict. They will be kept as strings and handled accordingly as needed.

In [38]:
path = r'../data/generated/'
reviews_df.to_csv(path + 'reviews.csv', index=False)